# 他リポジトリから実験の入力データを用意する

[データガバナンス機能](http://dg02.dg.rcos.nii.ac.jp/)に公開されているリポジトリやあなたのリポジトリのデータをこの実験の入力データとして用意します。  
一括でファイルの実体をダウンロードするのではなく、必要なファイルのみファイルの実体をダウンロードするため、大容量データでも時間がかかりにくくなります。  
上から順番に実行してください。

※もし入力データとして利用したいファイルの実体が「git-annex」によってAWS S3準拠のオブジェクトストレージに保存されている場合、[こちら](util/operate_s3_annex.ipynb)を実行することで当該データを取得できます。

## 1. 入力データにするリポジトリのURLおよび実験名を入力する

以下のセルを実行して、入力データにしたいリポジトリのURLおよび実験名を入力してください。  
入力したURLの履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力に誤りがある場合は、再度このセルを実行して下さい。

In [ ]:
from ipywidgets import Text, Button, Layout
from IPython.display import clear_output
import datetime

def on_click_callback(clicked_button: Button) -> None:
    global input_repo
    input_repo = text.value
    clear_output()
    print("入力完了しました：", input_repo)

# テキストボックス
text = Text(
    description='URL：',
    placeholder='http://dg02.dg.rcos.nii.ac.jp/user/repository_title.git',
    layout=Layout(width='500px')
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

In [ ]:
from ipywidgets import Text, Button
from IPython.display import clear_output

def on_click_callback(clicked_button: Button) -> None:
    global input_experiment
    input_experiment = text.value
    clear_output()
    print("入力完了しました：", input_experiment)

# テキストボックス
text = Text(
    description='実験名：'
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text.on_submit(on_click_callback)
display(text, button)

## 2. リポジトリのディレクトリ階層をダウンロードする

In [ ]:
from util.scripts import utils

# URLからリポジトリ名を抽出する
input_repo_title = input_repo.split('/')[-1].replace('.git', '')
tmp_path = '/home/jovyan/.tmp/' + input_repo_title

# リポジトリ名のフォルダが無ければ作成する
!mkdir -p $tmp_path

# 入力データリポジトリをサブデータセットとしてクローンする
%cd ~/
!datalad clone -d . $input_repo $tmp_path

## 3. ファイルの実体をダウンロードする

出力データ(output_dataフォルダ配下)全ての実体をダウンロードしたい場合は`3.1`を、  
特定のファイルを選んで実体をダウンロードしたい場合は`3.2`を実行してください。

### 3.1 出力データ(output_dataフォルダ配下)の実体をダウンロードする

指定したリポジトリの出力データ(output_dataフォルダ配下)全ての実体をダウンロードしたい場合に実行してください。

In [ ]:
from util.scripts import utils

# 実験パッケージパスを取得する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# SAVEのフォルダが無ければ作成する
save_path = experiment_path + '/input_data/' + input_experiment
output_path = save_path + '/output_data'
!mkdir -p $save_path

%cd $tmp_path/experiments/$input_experiment/output_data
!datalad get . -r
!cp -r $tmp_path/experiments/$input_experiment/output_data/. $output_path

%cd ~/
!rm -rf $tmp_path

### 3.2 特定のファイルを指定して実体をダウンロードする

出力データの特定のファイルや、出力データ以外のファイルをダウンロードしたい場合に、以下のセルの`"ファイルパス"`をダウンロードしたいファイルパスに書き換えてから実行してください。  
※複数ファイルの実体をダウンロードしたい場合は、`!datalad get "ファイルパス"`の処理を必要な数だけコピーしてください。

In [ ]:
from util.scripts import utils

# 実験パッケージパスを取得する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# SAVEのフォルダが無ければ作成する
save_path = experiment_path + '/input_data/' + input_experiment
output_path = save_path + '/output_data'
!mkdir -p $save_path

%cd $tmp_path/experiments/$input_experiment/output_data

# (例)
# !datalad get "sample_a.csv"
# !datalad get "b/sample_b.csv"

!datalad get "ファイルパス"
!cp -r $tmp_path/experiments/$input_experiment/output_data/. $output_path

%cd ~/
!rm -rf $tmp_path

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import papermill as pm

%cd ~/
save_path = [ save_path ]
pm.execute_notebook(
    'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_入力データの準備', PATH = save_path, IS_RECURSIVE = False)
)

## 6. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  